In [13]:
import json
import yaml
import io
import os
import pandas as pd


In [14]:
#f = open('./results/waypoint_maker.waypoint_clicker.json')
#inferred = json.load(f)
handwritten_models_dict = dict()


rq1_results = { "" : {
    "periodic_found" : 0,
    "periodic_fp" : 0,
    "periodic_not_found" : 0,
    "reactive_found" : 0,
    "reactive_fp" : 0,
    "reactive_not_found" : 0,
    "state_var_found" : 0,
    "state_var_unknown_autoware" : 0,
    "state_var_not_found" : 0,
    "state_var_fp" : 0,
    "state_transition_found" : 0,
    "state_transition_fp" : 0,
    "state_transition_not_found" : 0,
    }}


systems = ["autoware", "fetch", "turtlebot", "autorally", "husky"]
rq1_files = []
num_handwritten_models = {"" : 0}
num_compared_models = {"" : 0}
for system_name in systems:
    num_handwritten_models[system_name] = 0
    num_compared_models[system_name] = 0

    periodic_found = 0
    periodic_not_found = 0
    periodic_fp = set()
    reactive_found = 0
    reactive_not_found = 0
    reactive_fp = set()
    state_var_found = 0
    state_var_unknown_autoware = 0
    state_var_not_found = 0
    state_var_fp = set()
    state_transition_found = 0
    state_transition_fp = set()
    state_transition_not_found = 0
#for system_name in ["autorally"]:
    handwritten_models_path = f"../rostimer-evaluation/handwritten-models/{system_name}/"
    if not os.path.exists(handwritten_models_path):
        continue
    handwritten_models_files = os.listdir(handwritten_models_path)
    

    for model_file_name in handwritten_models_files:
        if ".ct." in model_file_name or ".td." in model_file_name:
            continue
        num_handwritten_models[system_name] += 1
        num_handwritten_models[""] += 1
        with open(handwritten_models_path + model_file_name, 'r') as model_file:
            model_yml = yaml.safe_load(model_file)
            handwritten_models_dict[model_yml["package"] + "-" + model_yml["node"]] = model_yml

            if model_yml["node"] == "sync_drivers" and model_yml["package"] == "sync":
                continue #this node does not build due to a coding error in the original project and is therefore excluded

            model_file_name = f'{model_yml["package"]}.{model_yml["node"]}.json'
            inferred_path = f'./results/{model_file_name}'
            if not os.path.exists(inferred_path):
                print(f"ERROR! Didn't find {inferred_path}")
                continue
            rq1_files.append(model_file_name)
            
            inferred = json.load(open(inferred_path))

            num_compared_models[system_name] += 1
            num_compared_models[""] += 1

            print(inferred_path)    
            for pb_inferred in inferred["periodic_behavior"]:
                pb_2_found = False
                for pb_2 in model_yml["periodic_behavior"]:
                    if pb_inferred["publisher"]["variable"] == pb_2["publisher"]["variable"]:
                        pb_2_found = True
                if pb_2_found == False:
                    w = "periodic false positive: " + str(pb_inferred) + inferred_path
                    print(w)
                    periodic_fp.add(w)
            for pb in model_yml["periodic_behavior"]:
                found = False
                for pb_inferred in inferred["periodic_behavior"]:                
                    if pb_inferred["publisher"]["variable"] == pb["publisher"]["variable"]:
                        found = True
                if found:
                    periodic_found += 1
                    pb_inferred_freq = pb_inferred["rate"]
                    pb_freq = pb["rate"]
                    print(f"inferred frequency: {pb_inferred_freq}, handwritten frequency: {pb_freq}")
                else:
                    print(f"Did not find periodic "+pb["publisher"]["variable"] + " in " + inferred_path)
                    periodic_not_found += 1
                
            for rb_inferred in inferred["reactive_behavior"]:
                rb_2_found = False
                for rb_2 in model_yml["reactive_behavior"]:
                    subname = "subscriber"
                    if "message-filter" in rb_2:
                        subname = "message-filter"
                    if "event" in rb_inferred or "event" in rb_2:
                        if "event" in rb_2 and "event" in rb_inferred:
                            if rb_2["publisher"]["variable"] == rb_inferred["publisher"]["variable"] and rb_2["event"] == rb_inferred["event"]:
                                rb_2_found = True
                            #else: 
                            #    print(f'{rb["event"]} == {rb_inferred["event"]} and {rb_2["publisher"]["variable"]} == {rb_inferred["publisher"]["variable"]}')
                        continue
                    if rb_2["publisher"]["variable"] == rb_inferred["publisher"]["variable"] and rb_2[subname]["callback"] == rb_inferred["subscriber"]["callback"]:
                        rb_2_found = True
                if rb_2_found == False:
                    w = "reactive false positive: " + str(rb_inferred) + inferred_path
                    print(w)
                    reactive_fp.add(w)
            for rb in model_yml["reactive_behavior"]:
                found = False
                print(rb)
                for rb_inferred in inferred["reactive_behavior"]:
                    subname = "subscriber"
                    if "message-filter" in rb:
                        subname = "message-filter"
                    if "event" in rb_inferred or "event" in rb:
                        if "event" in rb and "event" in rb_inferred:
                            if rb["publisher"]["variable"] == rb_inferred["publisher"]["variable"] and rb["event"] == rb_inferred["event"]:
                                found = True
                            else: 
                                print(f'{rb["event"]} == {rb_inferred["event"]} and {rb["publisher"]["variable"]} == {rb_inferred["publisher"]["variable"]}')
                        continue
                    if rb["publisher"]["variable"] == rb_inferred["publisher"]["variable"] and rb[subname]["callback"] == rb_inferred["subscriber"]["callback"]:
                        found = True
                if found:
                    reactive_found += 1
                else:
                    print(f"Did not find reactive "+str(rb) +  " in " + str(inferred["reactive_behavior"]) + " file: " + inferred_path)
                    reactive_not_found += 1
            for sv_inferred in inferred["potential_state_vars"]:
                found_sv_2 = False
                for sv_2 in model_yml["component_state_machine"]["state_variables"]:
                    if sv_2["name"] == sv_inferred["qualified_name"]:
                        found_sv_2 = True   
                if found_sv_2 == False:
                    w = "state var false positive:" + str(sv_inferred) + inferred_path
                    print(f"({system_name}) " + w)
                    state_var_fp.add(w)
            for sv in model_yml["component_state_machine"]["state_variables"]:
                found = False
                for sv_inferred in inferred["potential_state_vars"]:
                    if sv["name"] == sv_inferred["qualified_name"]:
                        found = True
                if found:
                    state_var_found += 1
                    if sv_inferred["initial-value"]["kind"] == "unknown":
                        state_var_unknown_autoware += 1
                        print("initial-value not found!")
                        print(sv_inferred)
                    else: 
                        print("initial-value found!")
                        print(sv_inferred)
                else:
                    print(f"Did not find state var ({model_yml['system']}) " +str(sv) +  " in " + str(inferred["potential_state_vars"]) + " file: " + inferred_path)
                    state_var_not_found += 1

            if "transitions" in model_yml["component_state_machine"]:
                for st_inferred in inferred["transitions"]:
                    if st_inferred["condition"] == "True" and len(st_inferred["state_changes"]) == 0:
                        continue
                    found_2 = False
                    for st_2 in model_yml["component_state_machine"]["transitions"]:
                        st_2 = st_2["trigger"]
                        if st_2["type"] == st_inferred["type"]:
                            if st_2["type"] == "message" and st_2["callback"] == st_inferred["callback"]:
                                found_2 = True
                            if st_2["type"] == "interval" and st_2["interval"] == st_inferred["interval"]:
                                found_2 = True
                    if found_2 == False:
                        w = "state transition false positive:" + str(st_inferred) + inferred_path
                        print(w)
                        state_transition_fp.add(w)
                for st in model_yml["component_state_machine"]["transitions"]:
                    print(st)
                    #if (("condition" not in st["trigger"]) or st["trigger"]["condition"] == "True") and (("state-changes" not in st) or len(st["state-changes"])==0):
                    #    continue
                    st = st["trigger"]
                    found = False
                    if "transitions" in inferred:
                        for st_inferred in inferred["transitions"]:
                            print(st_inferred)
                            if st["type"] == st_inferred["type"]:
                                if st["type"] == "message" and st["callback"] == st_inferred["callback"]:
                                    found = True
                                if st["type"] == "interval" and st["interval"] == st_inferred["interval"]:
                                    found = True
                                
                    if found:
                        state_transition_found += 1

                    else:
                        print(f"Did not find state transition "+str(st) +  " in " + str(inferred["transitions"]) + " file: " + inferred_path)
                        state_transition_not_found += 1                
    rq1_results[system_name] = {
    "periodic_found" : periodic_found,
    "periodic_not_found" : periodic_not_found,
    "periodic_fp" : len(periodic_fp),
    "reactive_found" : reactive_found,
    "reactive_not_found" : reactive_not_found,
    "reactive_fp" : len(reactive_fp),
    "state_var_found" : state_var_found,
    "state_var_unknown_autoware" : state_var_unknown_autoware,
    "state_var_not_found" : state_var_not_found,
    "state_var_fp" : len(state_var_fp),
    "state_transition_fp" : len(state_transition_fp),
    "state_transition_found" : state_transition_found,
    "state_transition_not_found" : state_transition_not_found,
    }
    rq1_results[""] = {
    "periodic_found" : periodic_found + rq1_results[""]["periodic_found"],
    "periodic_not_found" : periodic_not_found + rq1_results[""]["periodic_not_found"],
    "periodic_fp" : len(periodic_fp) + rq1_results[""]["periodic_fp"],
    "reactive_found" : reactive_found + rq1_results[""]["reactive_found"],
    "reactive_not_found" : reactive_not_found + rq1_results[""]["reactive_not_found"],
    "reactive_fp" : len(reactive_fp) + rq1_results[""]["reactive_fp"],
    "state_var_fp" : len(state_var_fp) + rq1_results[""]["state_var_fp"],
    "state_var_found" : state_var_found + rq1_results[""]["state_var_found"],
    "state_var_unknown_autoware" : state_var_unknown_autoware + rq1_results[""]["state_var_unknown_autoware"],
    "state_var_not_found" : state_var_not_found + rq1_results[""]["state_var_not_found"],
    "state_transition_found" : state_transition_found + rq1_results[""]["state_transition_found"],
    "state_transition_fp" : len(state_transition_fp) + rq1_results[""]["state_transition_fp"],
    "state_transition_not_found" : state_transition_not_found + rq1_results[""]["state_transition_not_found"],
    }




In [15]:
processed_models = []

In [16]:
rq2_results = {}
partial_models = set()
def rq2(inferred_models_files):

    rate_known = 0
    rate_unknown = 0

    reactive_known = 0
    reactive_unknown = 0

    state_var_unknown = 0
    state_var_known = 0

    rb_callback_unknown = 0
    rb_callback_known = 0

    state_transition_unknown = 0
    state_transition_known = 0

    processed = set()

    
    num_models_inferred = 0

    num_unclassified_publish = 0

    for model_file_name in inferred_models_files:

        if not ".json" in model_file_name:
            continue
        with open(inferred_models_path + model_file_name, 'r') as model_file:
            num_models_inferred += 1
            #print(model_file)
            inferred = json.load(model_file)
            #print(inferred)
            if "node" in inferred:
                node_name = inferred["node"] 
                package_name = inferred["package"] 
            else:
                node_name = model_file_name 
                package_name = model_file_name
            for pb_inferred in inferred["periodic_behavior"]:
                str_pb = str(pb_inferred)
                if str_pb in processed: 
                    continue
                processed.add(str_pb)
                #print(f"periodic in {model_file}")
                if pb_inferred["rate"]["kind"] == "unknown":
                    rate_unknown += 1
                    partial_models.add((package_name, node_name))
                    #print(f"Unknown rate ({package_name} : {node_name}): {pb_inferred}")
                    
                else: 
                    rate_known += 1
            for rb_inferred in inferred["reactive_behavior"]:

                reactive_known += 1

            for sv_inferred in inferred["potential_state_vars"]:
                str_pb = str(sv_inferred)
                if str_pb in processed: 
                    continue
                processed.add(str_pb)

                #print(f"State Var in {model_file}")
                if sv_inferred["initial-value"]["kind"] == "unknown":
                    state_var_unknown += 1
                    partial_models.add((package_name, node_name))
                    print(f"Unknown state var ({package_name} : {node_name}): " + str(sv_inferred))
                else: 
                    state_var_known += 1

            for rb_inferred in inferred["reactive_behavior"]:
                str_rb = str(rb_inferred)
                if str_rb in processed: 
                    continue
                processed.add(str_rb)

                #print(f"State Var in {model_file}")
                subname = "subscriber"
                if "event" in rb_inferred:
                    if "unknown" in rb_inferred["event"]:
                        rb_callback_unknown += 1
                    else:
                        rb_callback_known += 1
                    continue
                if "message-filter" in rb_inferred:
                    subname = "message-filter"
                if "unknown" in rb_inferred[subname]["callback"]:
                    rb_callback_unknown += 1
                    partial_models.add((package_name, node_name))
                    #print("reactive unknown: " + str(sv_inferred))
                else: 
                    rb_callback_known += 1

            for transition in inferred["transitions"]:
                for state_change in transition["state_changes"]:
                    state_change_str = str(transition) + str(state_change)
                    if state_change_str in processed: 
                        continue
                    processed.add(state_change_str)

                    #print(f"state_changes in {model_file}")
                    if "unknown" in state_change["new_value"]:
                        state_transition_unknown += 1
                        partial_models.add((package_name, node_name))
                        print(f"{system_name}: State Transitions unknown ({model_file_name})")
                        print(state_change)
                    else:
                        state_transition_known += 1
                        #print(f"{system_name}: State Transitions known ({model_file_name})")
                        #print(state_change)

            if "unclassified_publish" not in inferred:
                continue
            for unclassified_publish in inferred["unclassified_publish"]:
                num_unclassified_publish += 1
                #print(f"unclassified_publish in {model_file_name}: {unclassified_publish}")

    #print(f'len(inferred_models_files): {len(inferred_models_files)}')
    #print(f'rate_known: {rate_known} rate_unknown: {rate_unknown} ({100*rate_known/(rate_known + rate_unknown)}%)')

    #print(f'rb_callback_known: {rb_callback_known} rb_callback_unknown: {rb_callback_unknown} ({100*rb_callback_known/(rb_callback_known + rb_callback_unknown)}%)')
    #print(f'state_var_known: {state_var_known} state_var_unknown: {state_var_unknown} ({100*state_var_known/(state_var_known + state_var_unknown)}%)')

    #print(f'state_transition_known: {state_transition_known} state_transition_unknown: {state_transition_unknown} ({100*state_transition_known/(state_transition_known + state_transition_unknown)}%)')
    return { 
            "num_unclassified_publish" : num_unclassified_publish,
            "num_models_inferred" : num_models_inferred,
            "rate_known" : rate_known, 
            "rate_unknown" : rate_unknown, 
            "rb_callback_known" : rb_callback_known, 
            "rb_callback_unknown" : rb_callback_unknown, 
            "state_var_known" : state_var_known, 
            "state_var_unknown" : state_var_unknown,
            "state_transition_known" : state_transition_known,
            "state_transition_unknown" : state_transition_unknown
            }



inferred_models_dict = dict()
inferred_models_path = "./results/"
inferred_models_files = os.listdir(inferred_models_path)
rq2_results[""] = rq2(inferred_models_files)

print(inferred_models_files)



Unknown state var (velodyne_driver : driver_nodelet): {'kind': 'member-var-ref', 'qualified_name': 'velodyne_driver::VelodyneDriver::(anonymous struct)::npackets', 'type': 'integer', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'member-var-ref', 'qualified_name': 'velodyne_driver::VelodyneDriver::config_', 'type': 'unsupported', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'this-expr'}}}
Unknown state var (velodyne_driver : driver_nodelet): {'kind': 'member-var-ref', 'qualified_name': 'velodyne_driver::VelodyneDriver::(anonymous struct)::cut_angle', 'type': 'integer', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'member-var-ref', 'qualified_name': 'velodyne_driver::VelodyneDriver::config_', 'type': 'unsupported', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'this-expr'}}}
Unknown state var (rslidar_driver : rslidar_driver): {'kind': 'member-var-ref', 'qualified_name': 'rslidar_driver::rslidarDriver::(anonymous struct)::cut_angle', 'type': 'intege

In [17]:
df_all = None
df_crashed_all = None
for system_name in systems:
    path = f"./results/recovery/subjects/{system_name}/recovered-models.csv"
    df_ = pd.read_csv(path)
    df = df_[df_['crashed'] == False]
    df_crashed = df_[df_['crashed'] == True]
    df_crashed = df_crashed[["package", "node"]]
    time_taken = df["time_taken"].sum()
    print("\\newcommand\\total"+system_name.capitalize()+"ExecutionTimeSec{"+str(time_taken)+"}")
    print("\\newcommand\\avg"+system_name.capitalize()+"ComponentExecutionTime{"+str(df["time_taken"].mean())+"}")
    if df_all is None:
        df_all = df
        df_crashed_all = df_crashed
    else: 
        df_all = pd.concat([df_all, df])
        df_crashed_all = pd.concat([df_crashed_all, df_crashed]).drop_duplicates()
    
    paths = []
    for index, row in df.iterrows():
        paths.append(row["package"] + "."  + row["node"] + ".json")
    rq2_results[system_name] = rq2(paths)
    rq2_results[system_name]["crashed"] = len(df_crashed)
    rq2_results[system_name]["notcrashed"] = len(df)

print("\\newcommand\\avgComponentExecutionTime{"+str(df_all["time_taken"].mean())+"}")
rq2_results["CaseStudy"] = rq2(rq1_files)
rq2_results["CaseStudy"]["crashed"] = num_handwritten_models[system_name] - num_compared_models[system_name]
rq2_results["CaseStudy"]["notcrashed"] = num_compared_models[system_name]
rq2_results[""]["crashed"] = len(df_crashed_all)
rq2_results[""]["notcrashed"] = len(df_all)
    

\newcommand\totalAutowareExecutionTimeSec{13645.296455264557}
\newcommand\avgAutowareComponentExecutionTime{62.024074796657075}
Unknown state var (pos_db : pos_downloader): {'kind': 'variable-reference', 'qualified_name': 'itr', 'type': 'unsupported', 'initial-value': {'kind': 'unknown'}}
Unknown state var (pos_db : pos_downloader): {'kind': 'variable-reference', 'qualified_name': 'type', 'type': 'integer', 'initial-value': {'kind': 'unknown'}}
Unknown state var (rslidar_driver : rslidar_driver): {'kind': 'member-var-ref', 'qualified_name': 'rslidar_driver::rslidarDriver::(anonymous struct)::cut_angle', 'type': 'integer', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'member-var-ref', 'qualified_name': 'rslidar_driver::rslidarDriver::config_', 'type': 'unsupported', 'initial-value': {'kind': 'unknown'}, 'base': {'kind': 'this-expr'}}}
Unknown state var (velodyne_driver : velodyne_node): {'kind': 'member-var-ref', 'qualified_name': 'velodyne_driver::VelodyneDriver::(anonymous s

In [18]:
for system_name in [""]: #systems + [""]:
    if system_name == "husky":
        continue
    print(f"Periodic FP {system_name} "+str(rq1_results[system_name]["periodic_fp"]))
    print(f"Reactive FP {system_name} "+str(rq1_results[system_name]["reactive_fp"]))
    print(f"state_var_fp FP {system_name} "+str(rq1_results[system_name]["state_var_fp"]))
    print(f"state_transition_fp FP {system_name} "+str(rq1_results[system_name]["state_transition_fp"]))

    print(f"Periodic {system_name} "+str(rq1_results[system_name]["periodic_fp"]))
    print(f"Reactive {system_name} "+str(rq1_results[system_name]["reactive_fp"]))
    print(f"state_var_fp {system_name} "+str(rq1_results[system_name]["state_var_fp"]))
    print(f"state_transition_fp {system_name} "+str(rq1_results[system_name]["state_transition_fp"]))
    
    periodic_found = rq1_results[system_name]["periodic_found"]
    periodic_not_found = rq1_results[system_name]["periodic_not_found"]
    reactive_found = rq1_results[system_name]["reactive_found"]
    reactive_not_found = rq1_results[system_name]["reactive_not_found"]
    state_var_found = rq1_results[system_name]["state_var_found"]
    state_var_not_found = rq1_results[system_name]["state_var_not_found"]
    state_transition_found = rq1_results[system_name]["state_transition_found"]
    state_transition_not_found = rq1_results[system_name]["state_transition_not_found"]
    print(f'periodic_found: {periodic_found} periodic_not_found: {periodic_not_found} ({100* periodic_found/(periodic_found + periodic_not_found)}% of {periodic_found + periodic_not_found})')
    print(f'reactive_found: {reactive_found} reactive_not_found: {reactive_not_found} ({100*reactive_found/(reactive_found + reactive_not_found)}% of {reactive_found + reactive_not_found})')
    print(f'state_var_found: {state_var_found} state_var_not_found: {state_var_not_found} ({100 * state_var_found / (state_var_found + state_var_not_found)}% of {state_var_found + state_var_not_found}) state_var_unknown: {state_var_unknown_autoware}')
    print(f'state_transition_found: {state_transition_found} state_transition_not_found: {state_transition_not_found} ({100*state_transition_found/(state_transition_found + state_transition_not_found)}% of {state_transition_found + state_transition_not_found})')

Periodic FP  0
Reactive FP  0
state_var_fp FP  0
state_transition_fp FP  0
Periodic  0
Reactive  0
state_var_fp  0
state_transition_fp  0


ZeroDivisionError: division by zero

In [ ]:


for system_name in systems + [""]:
    if system_name == "husky":
        continue

    print("\\newcommand\\num"+system_name.capitalize()+"NodesHandwritten{"+str(num_handwritten_models[system_name])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"ComparedNodes{"+str(num_compared_models[system_name])+"}")


    print("\\newcommand\\num"+system_name.capitalize()+"Periodic{"+str(rq1_results[system_name]["periodic_found"] + rq1_results[system_name]["periodic_not_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"PeriodicFound{"+str(rq1_results[system_name]["periodic_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"PeriodicFP{"+str(rq1_results[system_name]["periodic_fp"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"PeriodicFoundAndFP{"+str(rq1_results[system_name]["periodic_found"] + rq1_results[system_name]["periodic_fp"])+"}")
    pacc = 100 * rq1_results[system_name]["periodic_found"] / ((rq1_results[system_name]["periodic_fp"] + rq1_results[system_name]["periodic_found"]) or 1)
    print("\\newcommand\\num"+system_name.capitalize()+"PeriodicAccuracy{"+str(pacc)+"}")

    print("\\newcommand\\num"+system_name.capitalize()+"Reactive{"+str(rq1_results[system_name]["reactive_found"] + rq1_results[system_name]["reactive_not_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"ReactiveFound{"+str(rq1_results[system_name]["reactive_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"ReactiveFP{"+str(rq1_results[system_name]["reactive_fp"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"ReactiveFoundAndFP{"+str(rq1_results[system_name]["reactive_found"]+rq1_results[system_name]["reactive_fp"])+"}")
    racc = 100 * rq1_results[system_name]["reactive_found"] / ((rq1_results[system_name]["reactive_fp"] + rq1_results[system_name]["reactive_found"]) or 1)
    print("\\newcommand\\num"+system_name.capitalize()+"ReactiveAccuracy{"+str(racc)+"}")


    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsFound{"+str(rq1_results[system_name]["state_var_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsUnknown{"+str(rq1_results[system_name]["state_var_unknown_autoware"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsNotFound{"+str(rq1_results[system_name]["state_var_not_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsFP{"+str(rq1_results[system_name]["state_var_fp"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsFoundAndFP{"+str(rq1_results[system_name]["state_var_found"] + rq1_results[system_name]["state_var_fp"])+"}")
    svacc = 100 * rq1_results[system_name]["state_var_found"] / ((rq1_results[system_name]["state_var_fp"] + rq1_results[system_name]["state_var_found"]) or 1)
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsAccuracy{"+str(svacc)+"}")

    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsFound{"+str(rq1_results[system_name]["state_transition_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsNotFound{"+str(rq1_results[system_name]["state_transition_not_found"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsFP{"+str(rq1_results[system_name]["state_transition_fp"])+"}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsFoundAndFP{"+str(rq1_results[system_name]["state_transition_found"] + rq1_results[system_name]["state_transition_fp"])+"}")
    stacc = 100 * rq1_results[system_name]["state_transition_found"] / ((rq1_results[system_name]["state_transition_fp"] + rq1_results[system_name]["state_transition_found"]) or 1)
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsAccuracy{"+str(stacc)+"}")


    print("\\newcommand\\"+system_name.capitalize()+"periodicPrecisionPercent{\\fpeval {100 * \\num"+system_name.capitalize()+"PeriodicFound  / \\num"+system_name.capitalize()+"Periodic} }")
    print("\\newcommand\\num"+system_name.capitalize()+"PeriodicNotFound{\\the\\numexpr \\num"+system_name.capitalize()+"Periodic - \\num"+system_name.capitalize()+"PeriodicFound \\relax}")
    print("\\newcommand\\"+system_name.capitalize()+"reactivePrecisionPercent{\\fpeval {100 * \\num"+system_name.capitalize()+"ReactiveFound  / \\num"+system_name.capitalize()+"Reactive} }")



    print("\\newcommand\\num"+system_name.capitalize()+"ReactiveNotFound{\\the\\numexpr \\num"+system_name.capitalize()+"Reactive - \\num"+system_name.capitalize()+"ReactiveFound \\relax}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVars{\\the\\numexpr \\num"+system_name.capitalize()+"StateVarsFound + \\num"+system_name.capitalize()+"StateVarsNotFound \\relax}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsEngineeringLimitationPercent{\\fpeval {100 * \\num"+system_name.capitalize()+"StateVarsNotFound  / \\num"+system_name.capitalize()+"StateVars} }")
    print("\\newcommand\\num"+system_name.capitalize()+"StateVarsEngineeringLimitationPrecision{\\fpeval {100 - \\num"+system_name.capitalize()+"StateVarsEngineeringLimitationPercent } }")
    

    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitions{\\the\\numexpr \\num"+system_name.capitalize()+"StateTransitionsFound + \\num"+system_name.capitalize()+"StateTransitionsNotFound \\relax}")
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsEngineeringLimitationPercent{\\fpeval {100 * \\num"+system_name.capitalize()+"StateTransitionsNotFound  / \\num"+system_name.capitalize()+"StateTransitions} }")
    print("\\newcommand\\num"+system_name.capitalize()+"StateTransitionsEngineeringLimitationPrecision{\\fpeval {100 - \\num"+system_name.capitalize()+"StateTransitionsEngineeringLimitationPercent } }")
    
    
for s in ["", "CaseStudy"] + systems:

    print("\\newcommand\\num"+s.capitalize()+"NodesInferredTotal{"+str(rq2_results[s]['num_models_inferred'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"RbCallbackKnown{"+str(rq2_results[s]['rb_callback_known'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"RbCallbackUnknown{"+str(rq2_results[s]['rb_callback_unknown'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"RateKnown{"+str(rq2_results[s]['rate_known'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"RateUnknown{"+str(rq2_results[s]['rate_unknown'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"StateVarKnown{"+str(rq2_results[s]['state_var_known'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"StateVarUnknown{"+str(rq2_results[s]['state_var_unknown'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"StateTransitionKnown{"+str(rq2_results[s]['state_transition_known'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"StateTransitionUnknown{"+str(rq2_results[s]['state_transition_unknown'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"UnclassifiedPublish{"+str(rq2_results[s]['num_unclassified_publish'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"NodesCrashedTotal{"+str(rq2_results[s]['crashed'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"NodesNotCrashedTotal{"+str(rq2_results[s]['notcrashed'])+"}")
    print("\\newcommand\\num"+s.capitalize()+"NodesTotal{"+str(rq2_results[s]['crashed'] + rq2_results[s]['num_models_inferred'])+"}")
    



    print("\\newcommand\\num"+s.capitalize()+"StateTransitionKnownAndUnknown{\\the\\numexpr \\num"+s.capitalize()+"StateTransitionKnown + \\num"+s.capitalize()+"StateTransitionUnknown \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"StateTransitionKnownPercent{\\fpeval {100 * \\num"+s.capitalize()+"StateTransitionKnown  / \\num"+s.capitalize()+"StateTransitionKnownAndUnknown} }")

    print("\\newcommand\\num"+s.capitalize()+"RateKnownAndUnknown{\\the\\numexpr \\num"+s.capitalize()+"RateKnown + \\num"+s.capitalize()+"RateUnknown \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"RbCallbackKnownAndUnknown{\\the\\numexpr \\num"+s.capitalize()+"RbCallbackKnown + \\num"+s.capitalize()+"RbCallbackUnknown \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"RateKnownPercent{\\fpeval {100 * \\num"+s.capitalize()+"RateKnown  / \\num"+s.capitalize()+"RateKnownAndUnknown} }")
    print("\\newcommand\\num"+s.capitalize()+"RbCallbackKnownPercent{\\fpeval {100 * \\num"+s.capitalize()+"RbCallbackKnown  / \\num"+s.capitalize()+"RbCallbackKnownAndUnknown} }")
    print("\\newcommand\\num"+s.capitalize()+"StateVarKnownAndUnknown{\\the\\numexpr \\num"+s.capitalize()+"StateVarKnown + \\num"+s.capitalize()+"StateVarUnknown \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"StateVarKnownPercent{\\fpeval {100 * \\num"+s.capitalize()+"StateVarKnown  / \\num"+s.capitalize()+"StateVarKnownAndUnknown} }")

    print("\\newcommand\\num"+s.capitalize()+"BehTotal{\\the\\numexpr \\num"+s.capitalize()+"RateKnownAndUnknown +  \\num"+s.capitalize()+"RbCallbackKnownAndUnknown +  \\num"+s.capitalize()+"UnclassifiedPublish \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"BehRecovered{\\the\\numexpr \\num"+s.capitalize()+"BehTotal -  \\num"+s.capitalize()+"UnclassifiedPublish \\relax}")
    print("\\newcommand\\num"+s.capitalize()+"PublishRecoveryRate{\\fpeval {100 * \\num"+s.capitalize()+"BehRecovered  / \\num"+s.capitalize()+"BehTotal} }")



In [ ]:
for (package, node) in partial_models:
    print(f"pipenv run rosdiscover recover <file> {package} {node}")

